<h1 style="text-align: center; font-size: 50px;"> Register Model </h1>

# Notebook Overview

- Configure the Environment
- Define Constants and Paths
- Load Configuratons and Secrets
- Register Model 
- Evaluate Model
- Log Execution Time

# Step 0: Configure the Environment

In [ ]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [ ]:
start_time = time.time()  

logger.info("Notebook execution started.")

In [ ]:
# Standard Libraries
import logging
import sys
import os
import json
from pathlib import Path

# Add src directory to system path
sys.path.append(str(Path("..").resolve() / "src"))

# Internal Modules
from markdown_correction_service import MarkdownCorrectionService
from utils import load_config_and_secrets
from llm_metrics import (
    semantic_similarity_metric,
    grammar_error_count_metric,
    grammar_error_rate_metric,
    grammar_improvement_metric,
    grammar_score_metric,
    readability_improvement_metric,
    llm_judge_metric_local,
)

# Other modules
import pandas as pd
import mlflow
from mlflow.models import evaluate
from mlflow.metrics import (
    ari_grade_level,
    exact_match,
    rouge1,
    rougeL
)

# Define Constants and Paths

In [ ]:
CONFIG_PATH = Path("../configs/configs.yaml")
SECRETS_PATH = Path("../configs/secrets.yaml")
LOCAL_MODEL_PATH = Path("/home/jovyan/datafabric/llama3.1-8b-instruct/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf")

# Load Configurations and Secrets

In [ ]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

# Step 1: Register Model

In [ ]:
%%time

# Set MLflow experiement name
mlflow.set_experiment("markdown-correction-experiment")

# Start a new MLflow run for logging and model tracking
with mlflow.start_run(run_name="markdown-correction-run") as run:
    MarkdownCorrectionService.log_model(
        llm_artifact=str(LOCAL_MODEL_PATH),
        config_yaml=str(CONFIG_PATH),
        secrets_yaml=str(SECRETS_PATH),
    )

    # Create a model URI that poiints to the newly logged model in this run
    model_uri = f"runs:/{run.info.run_id}/markdown_corrector"

    # Register the logged model in the MLflow Model Registry
    mlflow.register_model(model_uri, "MarkdownCorrector")
    logger.info(f"Model registered: MarkdownCorrector")

# Step 2: Evaluate Model

In [ ]:
%%time

# Load evaluation results from the JSON file
with open("results.json", "r") as f:
    results = json.load(f)

# Extract original markdown texts from results
original_texts = [item["original"] for item in results]

# Create evaluation DataFrame
eval_df = pd.DataFrame(original_texts, columns=["markdown"])

# Run evaluation
results = mlflow.evaluate(
    model=model_uri,
    data=eval_df,
    targets="markdown",
    feature_names=["markdown"],
    extra_metrics=[
        ari_grade_level(),
        exact_match(),
        rouge1(),
        rougeL(),
        semantic_similarity_metric,
        grammar_error_count_metric,
        grammar_error_rate_metric,
        grammar_improvement_metric,
        grammar_score_metric,
        readability_improvement_metric,
        llm_judge_metric_local
    ]
)

# Print results (optional)
'''
logger.info("Evaluation results:")
logger.info(results.metrics)
'''

# Log evaluation metric results to MLflow for tracking
mlflow.log_metrics(results.metrics)

# Log Execution Time

In [ ]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")